<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js054_diffusionLJMD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [ ]:
#@title js054_diffusionLJMD2D / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js054_diffusionLJMD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js054_diffusionLJMD2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.09.25 created, last updated on 2018.11.22
//    ver 0.0.1  2019.01.18 v1, last updated on 2021.06.14
//    ver 0.0.2  2021.11.02 v2, last updated on 2021.11.02
//    ver 0.0.3  2023.04.03 v3, last updated on 2023.08.27
//
// --------------------  molecular dynamics 2D
//
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//    potential: Lennard-Jones V(r) = 4*epsilon*((sigma/r)^12-(sigma/r)^6)
//           force F(r) = -dV(r)/dr = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
//
//    for fast calculation
//      ignore F(r) r>rCutoff
//      force F(r) <- force table + linear interpolation (see setForceTable() and cutoff(r))
//      register near particles reg[][] (see registerNearParticles()), near means r<rCutoff+20*2000*dt
//        reg[][] use 20 times, assuming particle max speed < 2000m/s
//      force calculation: sum up force(r) (r<rCutoff)
//
// --------------------
*/

const diffusionLJMD2D = (function(){ // ====================  diffusionLJMD2D Module  ====================

	const g_AMU = 1.66053904e-27;			// (kg) atomic mass unit
	const g_kB = 1.380649e-23;				// (J/K) Boltzmann's constant
	const g_nMax = 8000;					// array max

	let g_molecKind = 2;					// 0:He,1:Ne,2:Ar,3:Kr,4:Xe,5:Hg
	let g_nParticles = 100;					// number of particles
	let g_sysTime = 0.0;					// (s) system time
	let g_timeStep = 10.0*1.0e-15;			// (s) time step dt
	let g_xMax = 40.0E-9;					// (m) x-Box size
	let g_yMax = 20.0E-9;					// (m) y-Box size
	let g_Nsx = 100;						// use pre-registration section(0 to Nsx,0 to Nsx)
	let g_Nsy = 50;							// use pre-registration section(0 to Nsx,0 to Nsy)
	let g_rCutoff = 1.0e-9;					// (m) force cutoff length
	let g_hh = 1.0e-12;						// (m) forceTable r-division
	let g_kineticEnergy = 0.0;				// (J) total kinetic energy
	let g_potentialEnergy = 0.0;			// (J) total potential energy
	let g_sysTemp = 0.0;					// (K) system temperature
	let g_meanTemp = 0.0;					// (K) averaged system temperature

	const g_xx = dim1( g_nMax );			// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );			// (m) y-component of i-th particle position
	const g_vx = dim1( g_nMax );			// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );			// (m/s) y-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );			// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );			// (N) y-component of total force applied i-th particle
	const g_potentialTable = dim1( 1010 );	// (J) potential table [V[0], V[hh], V[2hh],..., V[rCutoff]]
	const g_forceTable = dim1( 1010 );		// (N) force table(ki,kj) [F[0], F[hh], F[2hh],..., F[rCutoff]]
	const g_reg = dimInt2( g_nMax, 100 );	// reg[i][0]: total number of particles near i-th particle
											// reg[i][j]: particle number near i-th particle, (j>0)
	const g_section = dimInt3( 100, 100, 20 ); // section[i][j][0]: total number of particles in section[i][j]
											// section[i][j][k]: k-th particle number in section[i][j]
	const g_mark = dimInt1( g_nMax );		// 1:left hand side at t==0, 0:else
	const g_density = dimInt1( 20 );		// density[i] : number of particles in [i*xMax/20,(i+1)*xMax/20)

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dimInt3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Int32Array( nk );
			}
		}
		return a;
	}


	// --------------------  Lennard-Jones potential data  --------------------

	let g_mass = 39.948*g_AMU;		// (kg) mass of Ar
	let g_sigma = 3.418e-10;		// (m) Lennard-Jones potential sigma for Ar
	let g_epsilon = 1.711e-21;		// (J) Lennard-Jones potential epsilon FOR Ar

	const g_LJMolec = [
	//    mass(kg)        E(J)         sigma(m)   string  color
		[   4.003*g_AMU,  10.2*g_kB, 2.576e-10, "He",  "#ee4444" ], // 0 He
		[  20.183*g_AMU,  36.2*g_kB, 2.976e-10, "Ne",  "#eeaa00" ], // 1 Ne
		[  39.948*g_AMU, 124.0*g_kB, 3.418e-10, "Ar",  "#aadd00" ], // 2 Ar
		[  83.500*g_AMU, 190.0*g_kB, 3.610e-10, "Kr",  "#44dd44" ], // 3 Kr
		[ 131.300*g_AMU, 229.0*g_kB, 4.055e-10, "Xe",  "#0088ee" ], // 4 Xe
		[ 200.590*g_AMU, 851.0*g_kB, 2.898e-10, "Hg",  "#8800ee" ], // 5 Hg
		[   2.016*g_AMU,  33.3*g_kB, 2.968e-10, "H2",  "#ff8800" ], // 6 H2
		[  28.013*g_AMU,  91.5*g_kB, 3.681e-10, "N2",  "#ff0000" ], // 7 N2
		[  31.999*g_AMU, 113.0*g_kB, 3.433e-10, "O2",  "#ff0088" ], // 8 O2
		[  18.015*g_AMU, 809.1*g_kB, 2.641e-10, "H2O", "#ff00ff" ], // 9 H2O
		[  16.043*g_AMU, 137.0*g_kB, 3.822e-10, "CH2", "#bb00ff" ], //10 CH4
		[  44.010*g_AMU, 190.0*g_kB, 3.996e-10, "CO2", "#6600ff" ], //11 CO2
		[  28.011*g_AMU, 110.0*g_kB, 3.590e-10, "CO",  "#0000ff" ]  //12 CO
	];

	const g_rCollision = [];		// (m) = sigma, V(sigma)==0.0
	const g_rBond = [];				// (m) bond length == 2.0*g_rBond[kind], == sigma*2^(1/6)
	const g_massOf = [];			// (kg) g_massOf[kind] : mass of kind
	const g_strOf = [];				// g_strOf[kind] : string of kind, such as "Ar"
	const g_colorOf = [];			// g_colorOf[kind] : color of kind

	(function() {
		const n=g_LJMolec.length;
		for (let i=0; i<n; i++) {
			g_rCollision[i] = g_LJMolec[i][2]*0.5;
			g_rBond[i] = g_LJMolec[i][2]*0.5*1.12246;
			g_massOf[i] = g_LJMolec[i][0];
			g_strOf[i] = g_LJMolec[i][3];
			g_colorOf[i] = g_LJMolec[i][4];
		}
	}());

	function colorStrOf(kind) {
		return "<span style='color:"+g_colorOf[kind]+"'>"+g_strOf[kind]+"</span>";
	}

	function setLSparam() {
		g_mass = g_LJMolec[g_molecKind][0];
		g_epsilon = g_LJMolec[g_molecKind][1];
		g_sigma = g_LJMolec[g_molecKind][2];
	}

	// --- set potentialTable[ir] and forceTable[ir]

	function setForceTable() {
		const hh=g_hh;
		for (let ir=1; ir<=1001; ir++) {
			const r = ir*hh;
			const ri = (g_sigma/r);
			const r6 = ri*ri*ri*ri*ri*ri;
			// V(r) = 4*g_epsilon*((g_sigma/r)^12-(g_sigma/r)^6)
			g_potentialTable[ir] = cutoff(r)*4.0*g_epsilon*r6*(r6-1.0);
		}
		g_potentialTable[0] = g_potentialTable[1] + g_potentialTable[2];
		for (let ir=1; ir<=1000; ir++) { // F(r) = - grad V(r)
			g_forceTable[ir] = -(g_potentialTable[ir+1] - g_potentialTable[ir-1])/(2.0*hh);
		}
		g_forceTable[1001] = -(0.0 - g_potentialTable[1000])/(2.0*hh);
		g_forceTable[0] = g_forceTable[1];
	}

	function cutoff(r) {
		let ret;
		if (r>0 && r<0.8*g_rCutoff) {
			ret = 1.0;
		} else if (r>=0.8*g_rCutoff && r<g_rCutoff ) {
			ret = 0.5+0.5*Math.cos(Math.PI*(r-0.8*g_rCutoff)/(0.2*g_rCutoff));
		} else {
			ret = 0.0;
		}
		return ret;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( kind, nn, contTemp ) {
		g_sysTime = 0.0;
		g_molecKind = kind
		setLSparam();
		setForceTable();
		g_nParticles = setParticles(nn);
		ajustVelocity(contTemp);
		clearMark();
	}

	function setParticles(nn) {
		const d0=2.0*g_rCollision[g_molecKind];
		let nParticles = nn;
		for (let j=0; j<nn; j++) {
			let loopCount = 0;
			let i;
			do {
				g_xx[j] = (g_xMax-1.2*d0)*Math.random() + 0.6*d0;
				g_yy[j] = (g_yMax-1.2*d0)*Math.random() + 0.6*d0;
				for (i=0; i<j; i++) {
					if ((g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]) < 1.2*d0*d0) break;
				}
				loopCount = loopCount + 1;
				if (loopCount>1000) break;
			} while (i<j);
			if (loopCount>1000) {
				nParticles = j - 1;
				break;
			}
		}
		for (let i=0; i<nParticles; i++) {
			g_vx[i] = 200.0*normalRandom3();
			g_vy[i] = 200.0*normalRandom3();
			g_ffx[i] = 0.0;
			g_ffy[i] = 0.0;
		}
		return nParticles;
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}

	function setMark() {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			g_mark[i] = (g_xx[i]<=0.5*g_xMax) ? 1 : 0;
		}
	}

	function clearMark() {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			g_mark[i] = 0;
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( tempMode, contTemp ) {
		const nn=g_nParticles;
		if (tempMode==1) ajustVelocity(contTemp);
		registration();
		for (let i=1; i<=20; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		g_sysTemp = g_kineticEnergy/(nn*g_kB); //2D: E/N=kT, 3D: E/N=(3/2)kT
		g_meanTemp = 0.9*g_meanTemp + 0.1*g_sysTemp;
	}

	function moveParticles(dt) {
		const nn=g_nParticles;
		let ss = 0.0;
		const a = 0.5*dt/g_mass;
		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}
	}

	function calcForce() {
		const nn=g_nParticles, s05 = 0.5*g_sigma;
		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0;g_ffy[i]=0;
		}
		for (let i=0; i<nn-1; i++) {
			for (let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				const xij=g_xx[i]-g_xx[j], yij=g_yy[i]-g_yy[j];
				const rij = Math.sqrt(xij*xij+yij*yij);
				if (rij<g_rCutoff) {
					const f = force(rij);
					const fxij = f*xij/rij, fyij = f*yij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
				}
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
		}
	}

	function force(r) { // forceTable - linear interporation
		const hh=g_hh;
		const ir = Math.floor(r/hh);
		const a = r - ir*hh;
		g_potentialEnergy += ((hh-a)*g_potentialTable[ir] + a*g_potentialTable[ir+1])/hh;
		return ((hh-a)*g_forceTable[ir] + a*g_forceTable[ir+1])/hh;
	}

	function boundaryForce(r) {
		const ri = (3.418e-10/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*1.711e-21*r6*(r6-1.0); // epsilonOfAr = 1.711e-21
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	// --- registration with preRegistration

	function registration() {
		const nn=g_nParticles, nsx=g_Nsx, nsy=g_Nsy;
		preRegistration();
		const rreg = g_rCutoff+20*2000*g_timeStep;
		const rreg2 = rreg*rreg;
		for (let ipp=0; ipp<nn-1; ipp++) {
			let kp = 1;
			let i0 = Math.floor(nsx*(g_xx[ipp]-rreg)/g_xMax);
			if (i0<0) i0 = 0;
			let i1 = Math.floor(nsx*(g_xx[ipp]+rreg)/g_xMax );
			if (i1>=nsx) i1 = nsx-1;
			let j0 = Math.floor(nsy*(g_yy[ipp]-rreg)/g_yMax );
			if (j0<0) j0 = 0;
			let j1 = Math.floor(nsy*(g_yy[ipp]+rreg)/g_yMax );
			if (j1>=nsy) j1 = nsy-1;
			for (let i=i0; i<=i1; i++) {
				for (let j=j0; j<=j1; j++) {
					for (let iq=1; iq<=g_section[i][j][0]; iq++) {
						const jp = g_section[i][j][iq];
						if (jp>ipp) {
							const r2 = (g_xx[ipp]-g_xx[jp])*(g_xx[ipp]-g_xx[jp])+(g_yy[ipp]-g_yy[jp])*(g_yy[ipp]-g_yy[jp]);
							if (r2<rreg2) {
								g_reg[ipp][kp] = jp;
								kp = kp + 1;
							}
						}
					}
				}
			}
			g_reg[ipp][0] = kp;
		}
	}

	function preRegistration() {
		const nn=g_nParticles, nsx=g_Nsx, nsy=g_Nsy;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				g_section[i][j][0] = 0;
			}
		}
		for (let ipp=0; ipp<nn; ipp++) {
			let i = Math.floor(nsx*g_xx[ipp]/g_xMax);
			if (i>=nsx) i = nsx-1;
			let j = Math.floor(nsy*g_yy[ipp]/g_yMax);
			if (j>=nsy) j = nsy-1;
			const iq = g_section[i][j][0] + 1;
			g_section[i][j][0] = iq;
			g_section[i][j][iq] = ipp
		}
	}

	function maxNearParticles() {
		const nn=g_nParticles;
		let mx=0;
		for (let i=0; i<nn-1; i++) {
			if (mx<g_reg[i][0]) mx = g_reg[i][0];
		}
		return (mx-1);
	}

	// --- utility

	function systemTemperature() {
		const nn=g_nParticles;
		let ek=0.0; //kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return ek/(nn*g_kB);
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
		}
	}

	function setDensity() {
		const nn=g_nParticles;
		for (let i=0; i<20; i++) {
			g_density[i] = 0;
		}
		for (let i=0; i<nn; i++) {
			if (g_mark[i]==0) continue;
			let ii = Math.floor(20*g_xx[i]/g_xMax);
			if (ii>=20) ii = 19;
			g_density[ii] += 1;
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( kind, nn, contTemp )
		evolve:			timeEvolution,			// timeEvolution( tempMode, contTemp)

		setTemp:		ajustVelocity,			// ajustVelocity( temp )
		setDensity:		setDensity,				// setDensity() --- x-density
		setMark:		setMark,				// setMark()

		getSysParam:	function() { return [ g_molecKind, g_nParticles, g_timeStep, g_xMax, g_yMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getMolecStr:	function(kind) { return g_strOf[kind]; },
		getMolecData:	function(i) { const kind = g_molecKind; return [ g_rCollision[kind], g_colorOf[kind] ]; },
		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i] ]; },
		getMark:		function(i) { return g_mark[i]; },
		getDensity:		function(ix) { return g_density[ix]; },
	};

})(); // ====================  diffusionLJMD2D end  ====================


const js054 = (function(){ // ====================  js Module  ====================

	const theModule = diffusionLJMD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 320;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 2;			// 1:Ne  2:Ar  3:Kr  4:Xe  5:Hg
	let v_nn = 2000;			// number of molecules
	let v_nCalc = 2;
	let v_tempMode = 1;			// 0: adiabatic, 1: temp.control
	let v_contTemp = 300.0;

	let p_kind, p_nParticles, p_timeStep, p_xMax, p_yMax; // = theModule.getSysParam();
	let sysTime, temperature, kineticEnergy, potentialEnergy;
  let xxList = [];
	let yyList = [];
	let vxList = [];
	let vyList = [];

	let dispMode = 1;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let markFlag = false;
  let breakFlag = false;
  let perticleFlag = true;

	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_nn, v_contTemp ); // ( theme, nn, contTemp )
			[ p_kind, p_nParticles, p_timeStep, p_xMax, p_yMax ] = theModule.getSysParam();
			theModule.setMark();
      let perticleFlag = true;
		}

		if ( markFlag ) {
			markFlag = false;
			theModule.setMark()
		}

		if ( !pauseFlag ) {
				theModule.evolve( v_tempMode, v_contTemp );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_tempMode, v_contTemp );
		}

    if ( perticleFlag ) setParticlesData();

		draw( ctx, dispMode );

		requestAnimationFrame(animate);
	}

  function setParticlesData() {
    for (let i=0; i<p_nParticles; i++) {
			let x, y, vx, vy;
			[ x, y ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
			[ vx, vy ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
		}
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const xp = 40, yp = 20, xSize = 400, yTextPos = yCanvasSize-50;
		const ySize = xSize*p_yMax/p_xMax, scale = xSize/p_xMax;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		if ( dispMode==0 ) { // marked molec (red)
			drawBalls(ctx, xp, yp, scale );
		} else if ( dispMode==1 ) { // marked molec + x-distribution
			drawBalls(ctx, xp, yp, scale );
			plotDensity( ctx, xp, yp, xSize );
		} else if ( dispMode==2 ) { // x-distribution
			plotDensity( ctx, xp, yp, xSize );
		}


		// caption
		[ sysTime, temperature, kineticEnergy, potentialEnergy ] = theModule.getNow();
		const molecStr = theModule.getMolecStr(p_kind);
		ctx.fillStyle = "#888888";
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, xp, yTextPos);
		ctx.fillText(`Temp = ${temperature.toFixed(1)} (k)`, xp+200, yTextPos);
		ctx.fillText(`molec:${molecStr}, N = ${p_nParticles}`, xp, yTextPos+20);
		ctx.fillText(`cont. Temp = ${v_contTemp.toFixed(0)} (K)`, xp+200, yTextPos+20);
		ctx.fillText(`Box = ${(p_xMax*1.0e9).toFixed(1)}x${(p_yMax*1.0e9).toFixed(1)} (nm)`, xp, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, xp+200, yTextPos+40);
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"
	}

	function drawBalls( ctx, xp, yp, scale ) {
		const nParticles = p_nParticles, yMax = p_yMax, th = 2.0*Math.PI;
		for (let i=0; i<nParticles; i++) {
			let x, y, r, color;
			[ x, y ] = theModule.getPosition(i);
			[ r, color ] = theModule.getMolecData(i);
			ctx.fillStyle = (theModule.getMark(i)!=1) ? color : "#ff4444";
			ctx.beginPath();
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, r*scale, 0, th, false);
			ctx.fill();
		}
	}

	function plotDensity( ctx, xp, yp, xSize ) {
		const sc = xSize/20;
		theModule.setDensity();
		ctx.strokeStyle = "#ffaaaa";
		ctx.beginPath();
		for (let i=0; i<20; i++) {
			ctx.lineTo( i*sc+xp+10, 200-1.6*theModule.getDensity(i)+yp );
		}
		ctx.stroke();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 1 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_disp").selectedIndex;
	}

	function setMark() {
		markFlag = true;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = 1 + theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetTempMode( mode ) {
    v_tempMode = mode;
    document.getElementById("slct_tempMode").selectedIndex = mode;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, temperature, kineticEnergy, potentialEnergy ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ xxList, yyList, vxList, vyList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setDispMode:	setDispMode,	// setDispMode()
		setMark:		setMark,		// setMark()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetTempMode: pysetTempMode, // pysetTempMode( mode )
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, temperature, kineticEnergy, potentialEnergy ]
    pygetParticlesList, pygetParticlesList, //() :return [ xxList, yyList, vxList, vyList ]
	};

})(); // ====================  js054 module end  ====================


const js = js054;
//window.addEventListener('load', js.main );
js.main();

// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js054] diffusion - Lennard-Jones molecular dynamics 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="320"></canvas>
<br>

<label>molecule:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>Ne</option><option selected>Ar</option><option>Kr</option><option>Xe</option><option>Hg</option>
</select>
    <span style="margin-right: 50px;"></span>
<button onClick="js.setMark()">re-mark</button>
    <span style="margin-right: 80px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option selected>Temp.control</option>
</select>
<br>

<label>cont. Temp</label><label id="text_temp"> 300</label>
<input type="range" id="range_temp" min="10" max="600" value="300" step="2"
style="width:360px" oninput="js.setContTemp()">
<br>

<label>disp mode:</label>
<select id="slct_disp" onChange="js.setDispMode()">
<option>marked molec (red)</option>
<option selected>marked molec + x-distribution</option>
<option>x-distribution</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# python control
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print( f'i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps), temp = {sysTemp:>6.1f} (K), energy = {energy:9.6e} (J)' )
  time.sleep(3)
#

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / change theme

import time

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# reset
eval_js( 'js.reset()' )
print("-- reset --")

# change theme
themeList = [ 'Ne', 'Ar', 'Kr', 'Xe', 'Hg' ]
for theme in [ 0, 1, 2, 3, 4 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  for i in range(2):
    [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
    energy = kineticEnergy + potentialEnergy
    print( f'\t i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps), temp = {sysTemp:>6.1f} (K), energy = {energy:9.6e} (J)' )
    time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / change temperature

import time

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(3)

# change tempMode --> 0  # ( 0:adiabatic, 1:Temp.control )
eval_js( 'js.pysetTempMode(0)' )
print("-- tempMode: adiabatic --")

# change temperature
for contTemp in [ 200, 400, 600 ]:
  eval_js( 'js.setMark()' ) # set mark
  eval_js( 'js.pysetTemperature({})'.format(contTemp) ) # set temp
  print( f'-- set temperature :{contTemp} (K) --' )
  for i in range(4):
    [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
    energy = kineticEnergy + potentialEnergy
    print( f'-- -- i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps), temp = {sysTemp:>6.1f} (K), energy = {energy:9.6e} (J)' )
    time.sleep(3)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / plot vx, vy ( velocity space )

import time
import matplotlib.pyplot as plt

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print( f'i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps), temp = {sysTemp:>6.1f} (K), energy = {energy:9.6e} (J)' )
  time.sleep(1)

# get particle dara and plot
[ xxList, yyList, vxList, vyList ] = eval_js( 'js.pygetParticlesList()' )
print("-- plot vx, xy --", ", N = ", len(vxList) )
plt.figure(figsize=(6,6))
plt.title("velocity space")
plt.scatter( vxList, vyList )
plt.xlabel("vx")
plt.ylabel("vy")
plt.grid()
plt.show()

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")